In [13]:
import pandas as pd
import pickle

# Custom
from processing import tag_utterances
from processing import load_sem_types
from processing import DataPipeline
pd.set_option('display.max_columns', 500) # more columns displayed at once

There is a lot that could be done with formatting the data for training conversations:

* Option 1: All responses are equal
    * Treat every thread as a conversation
    * Every comment in the thread as a response to the original AskDocs

## Option 1: All responses are equal

* Each title of the thread is listed as the question 
* Every comment in that thread is listed as the answer
* The comment of the first query/post asked by the original author is ignored

In [2]:
data_instance = DataPipeline(comments_path = '../data/reddit_comments_askDocs_2014_to_2018_03.gz',
                            posts_path = '../data/original_posts_under_askDocs_subreddit_id.gz')
df = data_instance.load_full_thread()

print('Count of threads')
df['is_thread_start'].value_counts()

Comments Table Shape: (557648, 24)
Posts table shape: (43615, 35)
30710
Final combined table shape: (139535, 28)
Count of threads


0.0    108825
1.0     30710
Name: is_thread_start, dtype: int64

In [8]:
%%time
list_of_threads = df['link_id_short'].unique().tolist()

query = []
answer = []

# loop through all threads
for thread in list_of_threads:
    try:
        df_subset = df.loc[df['link_id_short']==thread]
        # assert there is one poster
        assert sum(df_subset['is_thread_start'].unique()) == 1

        thread_author = str(df_subset.loc[df_subset['is_thread_start']==1]['author'].unique()[0]).strip()
        thread_question = str(df_subset.loc[df_subset['is_thread_start']==1]['body'][0]).strip()


        thread_title = df_subset.title[df_subset.title.notnull()][0]

        try:
            thread_title_short = df_subset.url.unique()[1].split('/')[-2]
        except:
            thread_title_short = df_subset.url.unique()[1]


        thread_readers = df_subset.loc[df_subset['parent_id_short']==thread].author.tolist()

        if False:
            print('thread:',thread)
            print('thread_author:',thread_author)
            print('url:',df_subset.url.unique()[1])
            print('thread_title short:',thread_title_short)
            print('thread_title:',thread_title)
            print('thread_readers:',thread_readers)

        for index,row in df_subset.loc[df_subset['is_thread_start']!=1].iterrows():
            query.append({'author':thread_author,'reader':row['author'],'utterance':thread_title})
            answer.append({'author':row['author'], 'reader': thread_author,'utterance':row['body']})
    except:
        print(thread)
        
assert len(query)==len(answer)

CPU times: user 6min 25s, sys: 2.38 s, total: 6min 27s
Wall time: 6min 28s


In [17]:
pickle.dump(zip(query,answer),open( '../data/all_responses_equal.p', "wb" ))

In [170]:
for idx, row in df_subset.iterrows():
    print(df_subset.title[df_subset.title.notnull()][0])
    #print(row['url'],row['body'])
    print()

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!

Out of hand tonsil infection. Help!



In [171]:
for idx in range(len(query)):
    print("Q:", query[idx])
    print("A:", answer[idx])
    print()
print('Number of Q/A:',len(query))

Q: {'author': 'dpeters14fuck', 'reader': 'ebast', 'utterance': 'Out of hand tonsil infection. Help!'}
A: {'author': 'ebast', 'reader': 'dpeters14fuck', 'utterance': 'Well, then just be careful and try to avoid any kind of abdominal trauma. Best of luck! Get better soon so you can enjoy your vacations :)'}

Q: {'author': 'dpeters14fuck', 'reader': 'dpeters14fuck', 'utterance': 'Out of hand tonsil infection. Help!'}
A: {'author': 'dpeters14fuck', 'reader': 'dpeters14fuck', 'utterance': "Thanks for the advice and well wishing! It's now wednesday and I've been all good at work so I guess as long as I'm not straining myself too much I'll be all good"}

Q: {'author': 'dpeters14fuck', 'reader': 'ebast', 'utterance': 'Out of hand tonsil infection. Help!'}
A: {'author': 'ebast', 'reader': 'dpeters14fuck', 'utterance': "You're welcome :) well, that's hard to say actually. Contact sports are like the classic thing you tell patients not to do because the spleen gets a little bigger during mono. I'

In [91]:
for index,row in df_subset.iterrows():
    print(row['body'])

I just got them today, and it was my first time wearing contacts in a few years. They weren't moving every time I blinked, but occasionally, maybe once every few minutes. Then after a while, I noticed it happening much less often, although it still happened occasionally. Maybe because I was distracted, I wasn't focusing on it as much. Maybe I was blinking less? I was playing a video game/watching TV for a while, and it was much better. 


Other than that, no reddness or discomfort from them. Tomorrow I'm going to take them for a spin during the day and wear them a little longer. (i'll bring my glasses just in case though).
Does it happen every time you blink? How many days have you been wearing them? They do take a while to get used to although if they are moving every time you blink they may not be the best fit for you. Astigmatism is harder to fit then regular prescriptions as the lens or the cornea is irregularly shaped. 

If your eyes become slightly red or irritated from the lense

## Option 2: Every comment is a question and answer